## CSc 8830: Computer Vision - Assignment 4

1. Implement an application (must run on web or as an app on mobile device) using the stereo camera where it will recognize, track and estimate dimensions (at least 2D) of any object within 3m distance and inside field-of-view to the camera. You can use barcodes or text recognition tools for identification. However, the entire object must be tracked (not just the barcode or text). Machine/Deep learning tools are NOT allowed.

In [1]:
import cv2
import math
import imutils
import numpy as np
import depthai as dai
from imutils import perspective
from imutils import contours
from scipy.spatial import distance as dist


# Function to detect QR codes in an image
def detect_qr_codes(frame):
    qcd = cv2.QRCodeDetector()
    ret_qr, decoded_info, points, _ = qcd.detectAndDecodeMulti(frame)
    if ret_qr:
        for s, p in zip(decoded_info, points):
            if s:
                # Draw rectangle
                frame = cv2.polylines(frame, [p.astype(int)], True, (0, 255, 0), 8)
                # Add text
                cv2.putText(frame, s, (int(p[0][0]), int(p[0][1])-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            else:
                color = (0, 0, 255)
                frame = cv2.polylines(frame, [p.astype(int)], True, color, 8)
    return frame

class ObjectTracker:
    def __init__(self):
        # Dictionary to store the center points of the tracked objects
        self.object_centers = {}
        # Counter to keep track of object IDs
        self.object_id_count = 0

    def update(self, frame):
        # Extract Region of Interest (ROI)
        roi = frame

        # Background subtractor for object detection
        object_detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

        # 1. Object Detection
        mask = object_detector.apply(roi)
        _, mask = cv2.threshold(mask, 254, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        detections = []
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 100:
                x, y, w, h = cv2.boundingRect(cnt)
                detections.append([x, y, w, h])

        # 2. Object Tracking
        tracked_boxes_ids = self._track_objects(detections)
        for box_id in tracked_boxes_ids:
            x, y, w, h, obj_id = box_id
            cv2.rectangle(roi, (x, y), (x + w, y + h), (0, 255, 0), 3)

        return roi

    def _track_objects(self, object_rectangles):
        # List to store object bounding boxes and IDs
        tracked_objects = []

        # Update center points of existing objects or assign new IDs
        for rect in object_rectangles:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Check if the object was detected previously
            object_detected = False
            for obj_id, center in self.object_centers.items():
                dist = math.hypot(cx - center[0], cy - center[1])

                # If distance is within a threshold, update center and ID
                if dist < 25:
                    self.object_centers[obj_id] = (cx, cy)
                    tracked_objects.append([x, y, w, h, obj_id])
                    object_detected = True
                    break

            # If new object detected, assign a new ID
            if not object_detected:
                self.object_centers[self.object_id_count] = (cx, cy)
                tracked_objects.append([x, y, w, h, self.object_id_count])
                self.object_id_count += 1

        # Clean up dictionary by removing unused IDs
        new_object_centers = {}
        for obj_bb_id in tracked_objects:
            _, _, _, _, obj_id = obj_bb_id
            center = self.object_centers[obj_id]
            new_object_centers[obj_id] = center

        # Update dictionary with used IDs
        self.object_centers = new_object_centers.copy()
        return tracked_objects


# Create object tracker instance
tracker = ObjectTracker()

def midpoint(ptA, ptB):
    return ((ptA[0] + ptB[0]) * 0.5, (ptA[1] + ptB[1]) * 0.5)

class ObjectDimensionMarker:
    def __init__(self, object_width_inches):
        self.object_width_inches = object_width_inches
        self.pixelsPerMetric = None

    def mark_object_dimensions(self, gray_frame):
        # convert the frame to grayscale and blur it slightly
        gray = cv2.GaussianBlur(gray_frame, (7, 7), 0)

        # perform edge detection, then perform a dilation + erosion to
        # close gaps in between object edges
        edged = cv2.Canny(gray, 50, 100)
        edged = cv2.dilate(edged, None, iterations=1)
        edged = cv2.erode(edged, None, iterations=1)

        # find contours in the edge map
        cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)
        cnts = imutils.grab_contours(cnts)

        # sort the contours from left-to-right
        (cnts, _) = contours.sort_contours(cnts)
        
        # Initialize result frame
        result_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)

        # loop over the contours individually
        for c in cnts:
            # if the contour is not sufficiently large, ignore it
            if cv2.contourArea(c) < 100:
                continue

            # compute the rotated bounding box of the contour
            box = cv2.minAreaRect(c)
            box = cv2.boxPoints(box) if imutils.is_cv2() else cv2.boxPoints(box)
            box = np.array(box, dtype="int")

            # order the points in the contour and draw the outline of the rotated bounding box
            box = perspective.order_points(box)
            cv2.drawContours(result_frame, [box.astype("int")], -1, (0, 255, 0), 2)

            # loop over the original points and draw them
            for (x, y) in box:
                cv2.circle(result_frame, (int(x), int(y)), 5, (0, 0, 255), -1)

            # unpack the ordered bounding box
            (tl, tr, br, bl) = box

            # compute midpoints
            (tltrX, tltrY) = midpoint(tl, tr)
            (blbrX, blbrY) = midpoint(bl, br)
            (tlblX, tlblY) = midpoint(tl, bl)
            (trbrX, trbrY) = midpoint(tr, br)

            # draw midpoints
            cv2.circle(result_frame, (int(tltrX), int(tltrY)), 5, (255, 0, 0), -1)
            cv2.circle(result_frame, (int(blbrX), int(blbrY)), 5, (255, 0, 0), -1)
            cv2.circle(result_frame, (int(tlblX), int(tlblY)), 5, (255, 0, 0), -1)
            cv2.circle(result_frame, (int(trbrX), int(trbrY)), 5, (255, 0, 0), -1)

            # draw lines between the midpoints
            cv2.line(result_frame, (int(tltrX), int(tltrY)), (int(blbrX), int(blbrY)),
                     (255, 0, 255), 2)
            cv2.line(result_frame, (int(tlblX), int(tlblY)), (int(trbrX), int(trbrY)),
                     (255, 0, 255), 2)

            # compute the Euclidean distance between the midpoints
            dA = dist.euclidean((tltrX, tltrY), (blbrX, blbrY))
            dB = dist.euclidean((tlblX, tlblY), (trbrX, trbrY))

            # if the pixels per metric has not been initialized, then
            # compute it as the ratio of pixels to supplied metric
            if self.pixelsPerMetric is None:
                self.pixelsPerMetric = dB / self.object_width_inches

            # compute the size of the object in centimeters
            dimA_cm = dA / self.pixelsPerMetric * 2.54  # converting inches to centimeters
            dimB_cm = dB / self.pixelsPerMetric * 2.54

            # draw the object sizes on the image
            cv2.putText(result_frame, "{:.1f}cm".format(dimA_cm),
                        (int(tltrX - 15), int(tltrY - 10)), cv2.FONT_HERSHEY_SIMPLEX,
                        0.65, (255, 255, 255), 2)
            cv2.putText(result_frame, "{:.1f}cm".format(dimB_cm),
                        (int(trbrX + 10), int(trbrY)), cv2.FONT_HERSHEY_SIMPLEX,
                        0.65, (255, 255, 255), 2)

        return result_frame
    
# Initialize object dimension marker
object_marker = ObjectDimensionMarker(object_width_inches=2.24)

# Create a pipeline
pipeline = dai.Pipeline()

# Define a node for the stereo camera
stereo = pipeline.createStereoDepth()
stereo.setConfidenceThreshold(255)

# Define a node for the left camera
left = pipeline.createMonoCamera()
left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_720_P)

# Define a node for the right camera
right = pipeline.createMonoCamera()
right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_720_P)

# Connect left and right camera outputs to the stereo node
left.out.link(stereo.left)
right.out.link(stereo.right)

# Define a node for the output
xoutDepth = pipeline.createXLinkOut()
xoutDepth.setStreamName("depth")

# Link stereo camera output to the output node
stereo.depth.link(xoutDepth.input)

# Define a node to get left camera frames
xoutLeft = pipeline.createXLinkOut()
xoutLeft.setStreamName("left")

# Link left camera output to the output node
left.out.link(xoutLeft.input)

# Define a node to get right camera frames
xoutRight = pipeline.createXLinkOut()
xoutRight.setStreamName("right")

# Link right camera output to the output node
right.out.link(xoutRight.input)

# Define a source - color camera
cam = pipeline.createColorCamera()
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

# Create RGB output
xout = pipeline.createXLinkOut()
xout.setStreamName("rgb")
cam.video.link(xout.input)

# Connect to the device
with dai.Device(pipeline) as device:
    # Output queues
    depthQueue = device.getOutputQueue(name="depth", maxSize=4, blocking=False)
    leftQueue = device.getOutputQueue(name="left", maxSize=4, blocking=False)
    rightQueue = device.getOutputQueue(name="right", maxSize=4, blocking=False)
    rgbQueue = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

    # Start the pipeline
    device.startPipeline()

    # OpenCV setup
    camera_id = 0
    delay = 1
    window_name = 'Object Tracking & Recgnition through QR Code'
    cap = cv2.VideoCapture(camera_id)

    while True:
        # Get the depth frame
        inDepth = depthQueue.get()

        # Get the left camera frame
        inLeft = leftQueue.get()

        # Get the right camera frame
        inRight = rightQueue.get()

        # Get the rgb camera frame
        inSrc = rgbQueue.get()

        # Access the depth data
        depthFrame = inDepth.getFrame()

        # Access the left camera frame
        leftFrame = inLeft.getCvFrame()

        # Access the right camera frame
        rightFrame = inRight.getCvFrame()
        
        # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
        rgbFrame = inSrc.getCvFrame()

        '''# Resize frames if necessary to have the same size
        width = max(leftFrame.shape[1], rightFrame.shape[1])
        height = max(leftFrame.shape[0], rightFrame.shape[0])
        leftFrame = cv2.resize(leftFrame, (width, height))
        rightFrame = cv2.resize(rightFrame, (width, height))

        # Convert to anaglyph (red-cyan)
        anaglyph = np.zeros((height, width, 3), dtype=np.uint8)
        anaglyph[:, :, 0] = rightFrame[:, :]
        anaglyph[:, :, 1] = leftFrame[:, :]
        anaglyph[:, :, 2] = leftFrame[:, :]'''

        # Combine left and right frames horizontally
        stereoFrame = cv2.hconcat([leftFrame, rightFrame])

        # Detect QR codes in the stereo frame
        stereoFrame_with_qr = detect_qr_codes(stereoFrame)

        # Perform object detection and tracking
        stereoFrame_with_objects_qr = tracker.update(stereoFrame_with_qr)

        # Convert frame to grayscale
        gray_frame = cv2.cvtColor(rgbFrame, cv2.COLOR_BGR2GRAY)

        try:
            # Mark object dimensions
            marked_frame = object_marker.mark_object_dimensions(gray_frame)
        except:
            continue

        # Display stereo vision with QR code detection
        cv2.imshow("Stereo Vision (Right & Left)", stereoFrame_with_objects_qr)
        cv2.imshow("RGB Camera", marked_frame)
        #cv2.imshow("Stereoscopic Vision", anaglyph)
        #v2.imshow("Depth", depthFrame)

        # Exit loop by pressing 'q'
        if cv2.waitKey(1) == ord('q'):
            break

    # Release OpenCV resources
    cv2.destroyAllWindows()


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_5641/1840294533.py:203: DeprecationWarning: setConfidenceThreshold() is deprecated, Use 'initialConfig.setConfidenceThreshold()' instead
  stereo.setConfidenceThreshold(255)


[1844301031A5061300] [0.1.1] [0.925] [MonoCamera(1)] [error] OV7251 only supports THE_480_P/THE_400_P resolutions, defaulting to THE_480_P
[1844301031A5061300] [0.1.1] [0.926] [MonoCamera(2)] [error] OV7251 only supports THE_480_P/THE_400_P resolutions, defaulting to THE_480_P


/var/folders/cv/flgh8s7960bc7q380pyn0c6m0000gn/T/ipykernel_5641/1840294533.py:256: DeprecationWarning: Device(pipeline) starts the pipeline automatically. Use Device() and startPipeline(pipeline) otherwise
  device.startPipeline()


2. Use the DepthAI SDK or use ORB3-Visual SLAM (https://github.com/UZ-SLAMLab/ORB_SLAM3) to execute the scripts on your depth camera and run experiments in two different locations. Provide snapshots of your SLAM output and what limitations/corner cases do you observe.